In [1]:
import platform
platform.system()


'Windows'

In [15]:
!pip install lxml
!pip install shapely



   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 325.1 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/1.7 MB 751.6 kB/s eta 0:00:03
   --------------- ------------------------ 0.7/1.7 MB 3.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.2/1.7 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 6.0 MB/s eta 0:00:00


In [2]:
import os
import ctypes

dll_path = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\openslide-bin-4.0.0.8-windows-x64\bin"
os.add_dll_directory(dll_path)  # Python 3.8 이상에서만 사용 가능

# 혹은 수동으로 로드 (fallback)
# ctypes.WinDLL(os.path.join(dll_path, "libopenslide-0.dll"))





<AddedDllDirectory('C:\\Users\\roobe\\PKG - HER2 tumor ROIs_v3\\openslide-bin-4.0.0.8-windows-x64\\bin')>

In [5]:
#패치 나누는 코드-> xml파일에서 관심영역 지정해놓은 대로 폴리건으로 패치 나누고 0,1라벨링 라벨 파일에시켜서 파일이름-x,y로 저장->test/train set 나눠서 각각 하벨 파일 만들어서 저장


import openslide
from lxml import etree  #xml파일 여는데 필요
from shapely.geometry import Polygon, Point   #다각형 만들어줌
import os
from PIL import Image



# 설정
slide_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort\SVS"
output_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort\patches"
patch_size = 512
stride = 512

os.makedirs(output_dir, exist_ok=True)

annotations_dir= r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort\Annotations"
def parse_xml_polygon(xml_path):
    tree = etree.parse(xml_path) #부모 데이터
    root = tree.getroot()#트리 안에 있는 데이터를 가져옴
    namespace = root.nsmap.get(None, '')
    
    polygons = []
    for region in root.findall(".//Region"):#루트에서 지정된 영역 가져옴
        vertices = region.find(".//Vertices")#영역에서 꼭짓점 표시된 부분을 가져옴
        points = []
        for vertex in vertices.findall(".//Vertex"):
            x = float(vertex.attrib['X'])  #꼭짓점 부분의 x,y좌표 가져옴
            y = float(vertex.attrib['Y'])
            points.append((x, y))
        if len(points) > 2:
            polygons.append(Polygon(points)) #꼭짓점이 2개 이상일 떄 다각형 형성
    return polygons

def point_in_any_polygon(point, polygons): #각 폴리건 안에 위에서 지정한 포인트가 한 개라도 들어가는지 학인
    for poly in polygons:
        if poly.contains(point):
            return True
    return False

def process_slide(slide_path, xml_path, output_dir):#각 슬라이드에서 패치 자르기
    file_name = os.path.splitext(os.path.basename(slide_path))[0]
    slide = openslide.OpenSlide(slide_path)#오픈슬라이드로 svs 파일 열기
    polygons = parse_xml_polygon(xml_path)

    width, height = slide.dimensions #패치 크기 루프 돌릴 수 있도록 전체 크기 가져오기 

    for y in range(0, height - patch_size + 1, stride): #이렇게 잘라야 최종적으로 0-y파일 크기를 512씩 자를 수 있다
        for x in range(0, width - patch_size + 1, stride):
            center = Point(x + patch_size // 2, y + patch_size // 2)#패치의 중심 찾기
            if point_in_any_polygon(center, polygons): #위에 함수 써서 나눈 폴리건에 point가 들어가는지 확인-> 있으면 패치 형성
                patch = slide.read_region((x, y), 0, (patch_size, patch_size)).convert("RGB") #원보 슬라이드에서 x,y기준으로 패치를가져오고 색상지정, 0은 해상도
                patch_filename = f"{file_name}_{x}_{y}.png"
                patch.save(os.path.join(output_dir, patch_filename))

# 전체 반복
for fname in os.listdir(slide_dir):
    if fname.endswith(".svs"):
        slide_path = os.path.join(slide_dir, fname)
        xml_path = os.path.join(annotations_dir, os.path.splitext(fname)[0] + ".xml")
        
        if not os.path.exists(xml_path):
            print(f" XML 파일 없음: {xml_path}")  ##파일못찾은것같아요->경로 잘못침 수정완
            continue
        
        print(f" 처리 중: {fname}")
        process_slide(slide_path, xml_path, output_dir)

print(" 모든 슬라이드 패치 완료.")

 XML 파일 없음: C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort\Annotations\Her2Neg_Case_01.xml
 처리 중: Her2Neg_Case_02.svs
 처리 중: Her2Neg_Case_03.svs
 처리 중: Her2Neg_Case_04.svs
 처리 중: Her2Neg_Case_05.svs
 처리 중: Her2Neg_Case_06.svs
 처리 중: Her2Neg_Case_07.svs
 처리 중: Her2Neg_Case_08.svs
 처리 중: Her2Neg_Case_09.svs
 처리 중: Her2Neg_Case_10.svs
 처리 중: Her2Neg_Case_11.svs
 처리 중: Her2Neg_Case_12.svs
 처리 중: Her2Neg_Case_13.svs
 처리 중: Her2Neg_Case_14.svs
 처리 중: Her2Neg_Case_15.svs
 처리 중: Her2Neg_Case_16.svs
 처리 중: Her2Neg_Case_17.svs
 처리 중: Her2Neg_Case_18.svs
 처리 중: Her2Neg_Case_19.svs
 처리 중: Her2Neg_Case_20.svs
 처리 중: Her2Neg_Case_21.svs
 처리 중: Her2Neg_Case_22.svs
 처리 중: Her2Neg_Case_23.svs
 처리 중: Her2Neg_Case_24.svs
 처리 중: Her2Neg_Case_25.svs
 처리 중: Her2Neg_Case_26.svs
 처리 중: Her2Neg_Case_27.svs
 처리 중: Her2Neg_Case_28.svs
 처리 중: Her2Neg_Case_29.svs
 처리 중: Her2Neg_Case_30.svs
 XML 파일 없음: C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort\Annotations\Her2Neg_Case_31.xml
 처리 중: Her2Neg_

In [6]:
#neg1이 안되서 일단 수동으로 
slide_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort\SVS"
output_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort\patches"
patch_size = 512
stride = 512

os.makedirs(output_dir, exist_ok=True)

annotations_dir= r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort\Annotations\Her2Neg_Case_01.xml"
def parse_xml_polygon(xml_path):
    tree = etree.parse(xml_path) #부모 데이터
    root = tree.getroot()#트리 안에 있는 데이터를 가져옴
    namespace = root.nsmap.get(None, '')
    
    polygons = []
    for region in root.findall(".//Region"):#루트에서 지정된 영역 가져옴
        vertices = region.find(".//Vertices")#영역에서 꼭짓점 표시된 부분을 가져옴
        points = []
        for vertex in vertices.findall(".//Vertex"):
            x = float(vertex.attrib['X'])  #꼭짓점 부분의 x,y좌표 가져옴
            y = float(vertex.attrib['Y'])
            points.append((x, y))
        if len(points) > 2:
            polygons.append(Polygon(points)) #꼭짓점이 2개 이상일 떄 다각형 형성
    return polygons

def point_in_any_polygon(point, polygons): #각 폴리건 안에 위에서 지정한 포인트가 한 개라도 들어가는지 학인
    for poly in polygons:
        if poly.contains(point):
            return True
    return False

def process_slide(slide_path, xml_path, output_dir):#각 슬라이드에서 패치 자르기
    file_name = os.path.splitext(os.path.basename(slide_path))[0]
    slide = openslide.OpenSlide(slide_path)#오픈슬라이드로 svs 파일 열기
    polygons = parse_xml_polygon(xml_path)

    width, height = slide.dimensions #패치 크기 루프 돌릴 수 있도록 전체 크기 가져오기 

    for y in range(0, height - patch_size + 1, stride): #이렇게 잘라야 최종적으로 0-y파일 크기를 512씩 자를 수 있다
        for x in range(0, width - patch_size + 1, stride):
            center = Point(x + patch_size // 2, y + patch_size // 2)#패치의 중심 찾기
            if point_in_any_polygon(center, polygons): #위에 함수 써서 나눈 폴리건에 point가 들어가는지 확인-> 있으면 패치 형성
                patch = slide.read_region((x, y), 0, (patch_size, patch_size)).convert("RGB") #원보 슬라이드에서 x,y기준으로 패치를가져오고 색상지정, 0은 해상도
                patch_filename = f"{file_name}_{x}_{y}.png"
                patch.save(os.path.join(output_dir, patch_filename))

In [7]:
#라벨 나눈다음 train/0/, train/1/, val/0/, val/1/ 이런 식으로 자동 정리되게 해볼께요.
import os
import shutil

# 현재 패치 저장된 경로
patches_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort\patches"

# 라벨별 폴더 생성 (0 = neg, 1 = pos)
neg_dir = os.path.join(patches_dir, "0")
pos_dir = os.path.join(patches_dir, "1")
os.makedirs(neg_dir, exist_ok=True)
os.makedirs(pos_dir, exist_ok=True)

# 패치 파일 분류
for fname in os.listdir(patches_dir):
    if fname.endswith(".png"):
        src_path = os.path.join(patches_dir, fname)
        if "Pos" in fname:
            shutil.move(src_path, os.path.join(pos_dir, fname))
        elif "Neg" in fname:
            shutil.move(src_path, os.path.join(neg_dir, fname))

print("라벨별 폴더 정리가 완료되었습니다.")



라벨별 폴더 정리가 완료되었습니다.


In [10]:
import splitfolders
splitfolders.ratio(
    input= r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort\patches", 
    output=r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort", 
    seed=1337, 
    ratio=(.8, .2),
    move=True  # << 파일 이동
)


Copying files: 26989 files [00:20, 1316.50 files/s]
